In [ ]:
# ✅ 한글 폰트 설치 (NanumGothic)
!apt-get update -qq
!apt-get install -y fonts-nanum

# ✅ matplotlib 폰트 캐시 삭제
import os
os.system('rm -rf ~/.cache/matplotlib')

print("✅ 폰트 설치 완료! 이제 런타임 > 다시 시작(ctrl+M+.) 을 꼭 해주세요.")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 43 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (5,070 kB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 126281 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...
✅ 폰트 설치 완료! 이제 런타임 > 다시 시작(ctrl+M

In [ ]:
# catboost사용을 위한 설치(코랩엔 기본적으로 없음)
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 24.9 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection      import train_test_split, StratifiedKFold, cross_val_predict
from sklearn.preprocessing       import StandardScaler
from sklearn.impute              import SimpleImputer
from xgboost                     import XGBClassifier
from sklearn.metrics             import confusion_matrix, classification_report
from sklearn.pipeline            import Pipeline

from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix

# 경고 메시지 억제
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'catboost'

In [ ]:
# 데이터 로드 및 ID 제거
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FinalProject/tmp폴더/pca_features_with_segment_2.csv")
X = df.drop(columns=['Segment'])
y = df['Segment']

# E vs not-E 이진 타깃 생성
y_e = (y == 'E').astype(int)

# 학습/검증 분할 (20% 검증, E 분포 유지)
X_train, X_val, y_train_e, y_val_e, y_train, y_val = train_test_split(
    X, y_e, y, test_size=0.2, random_state=42, stratify=y_e
)

# 스케일러 학습 및 변환
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s   = scaler.transform(X_val)

In [ ]:
models = {
    'XGBoost': XGBClassifier(eval_metric='logloss', random_state=42, verbose=-1, verbosity=0, tree_method='gpu_hist', predictor = 'gpu_predictor'),
    'CatBoost': CatBoostClassifier(verbose=0, random_seed=42, task_type='GPU'),
    'LightGBM': LGBMClassifier(device='gpu', random_state=42),
    'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1)
}

In [ ]:
# 1. 모델1: E vs not-E 분류
model1 = CatBoostClassifier(verbose=0, random_seed=42, task_type='GPU')

# E vs not-E
model1.fit(X_train_s, y_train_e)

# 검증셋 E vs not-E
y1_pred = model1.predict(X_val_s)

print("=== 모델1: E vs not-E ===")
print(confusion_matrix(y_val_e, y1_pred))
print(classification_report(y_val_e, y1_pred, target_names=['not-E','E']))

# 검증셋 DataFrame 및 pred1 추가
df_val = pd.DataFrame(X_val_s, columns=X.columns)
df_val['true_seg'] = y_val.values
df_val['pred1']    = y1_pred

=== 모델1: E vs not-E ===
[[10636  5296]
 [ 2569 61499]]
              precision    recall  f1-score   support

       not-E       0.81      0.67      0.73     15932
           E       0.92      0.96      0.94     64068

    accuracy                           0.90     80000
   macro avg       0.86      0.81      0.83     80000
weighted avg       0.90      0.90      0.90     80000



In [ ]:
# 2. 모델2: A/B vs C/D 분류

# pred1 == 0 (not-E) 데이터
df_val_notE = df_val[df_val['pred1'] == 0].reset_index(drop=True)
X_val_notE  = df_val_notE[X.columns]

# A/B vs C/D 레이블 (A,B→1 / C,D→0)
y2_true = df_val_notE['true_seg'].apply(lambda s: 1 if s in ['A','B'] else 0)

# 학습셋 not-E 샘플 및 A/B vs C/D 타깃
mask_train_notE = (y_train_e == 0)
X_train_notE    = X_train_s[mask_train_notE]
y_train_notE    = y_train[mask_train_notE].apply(lambda s: 1 if s in ['A','B'] else 0)

# XGBoost 분류기
model2 = CatBoostClassifier(verbose=0, random_seed=42, task_type='GPU')

# A/B vs C/D
model2.fit(X_train_notE, y_train_notE)

# 검증셋 A/B vs C/D
y2_pred = model2.predict(X_val_notE)

print("\n=== 모델2: A/B vs C/D ===")
print(confusion_matrix(y2_true, y2_pred))
print(classification_report(y2_true, y2_pred, target_names=['C/D','A/B']))

# pred2 컬럼
df_val_notE['pred2'] = y2_pred


=== 모델2: A/B vs C/D ===
[[13167     1]
 [   34     3]]
              precision    recall  f1-score   support

         C/D       1.00      1.00      1.00     13168
         A/B       0.75      0.08      0.15        37

    accuracy                           1.00     13205
   macro avg       0.87      0.54      0.57     13205
weighted avg       1.00      1.00      1.00     13205



#### 모델2가 “A/B”로 예측한 6개 샘플에 대해서만 모델3을 돌려본 것

In [ ]:
# 모델3: A vs B 분류
# pred2 == 1 (A/B로 예측된) 데이터
df_val_AB = df_val_notE[df_val_notE['pred2'] == 1].reset_index(drop=True)
X_val_AB  = df_val_AB[X.columns]

# A vs B 레이블 (B→1 / A→0)
y3_true = df_val_AB['true_seg'].apply(lambda s: 1 if s == 'B' else 0)

# 학습셋 A/B 샘플 및 A vs B 타깃
mask_train_AB = mask_train_notE & (y_train.isin(['A','B']))
X_train_AB    = X_train_s[mask_train_AB]
y_train_AB    = y_train[mask_train_AB].apply(lambda s: 1 if s == 'B' else 0)

# XGBoost 분류기
model3 = CatBoostClassifier(verbose=0, random_seed=42, task_type='GPU')

# A vs B
model3.fit(X_train_AB, y_train_AB)

# 검증셋 A vs B
y3_pred = model3.predict(X_val_AB)

print("\n=== 모델3: A vs B ===")
print(confusion_matrix(y3_true, y3_pred))
print(classification_report(y3_true, y3_pred, labels=[0,1], target_names=['A','B']))


=== 모델3: A vs B ===
[[4]]
              precision    recall  f1-score   support

           A       1.00      1.00      1.00         4
           B       0.00      0.00      0.00         0

    accuracy                           1.00         4
   macro avg       0.50      0.50      0.50         4
weighted avg       1.00      1.00      1.00         4



#### 진짜 A/B인 37개 샘플에 대해서만 모델3을 돌려본 것

In [ ]:
# 실제 A/B 전체를 추출
df_val_trueAB = df_val_notE[df_val_notE['true_seg'].isin(['A','B'])]

X_val_trueAB = df_val_trueAB[X.columns]
y3_true_all  = df_val_trueAB['true_seg'].apply(lambda s: 1 if s == 'B' else 0)

# model3에 전체 A/B 데이터로 평가
y3_pred_all = model3.predict(X_val_trueAB)

print(confusion_matrix(y3_true_all, y3_pred_all))
print(classification_report(y3_true_all, y3_pred_all, labels=[0,1], target_names=['A','B']))

[[32  0]
 [ 5  0]]
              precision    recall  f1-score   support

           A       0.86      1.00      0.93        32
           B       0.00      0.00      0.00         5

    accuracy                           0.86        37
   macro avg       0.43      0.50      0.46        37
weighted avg       0.75      0.86      0.80        37



#### 모델2가 “C/D”로 예측한 6개 샘플에 대해서만 모델4를 돌려본 것

In [ ]:
# 4. 모델4: C vs D 분류
# pred2 == 0 (C/D로 예측된) 데이터
df_val_CD = df_val_notE[df_val_notE['pred2'] == 0].reset_index(drop=True)
X_val_CD  = df_val_CD[X.columns]

# C vs D 레이블 (D→1 / C→0)
y4_true = df_val_CD['true_seg'].apply(lambda s: 1 if s == 'D' else 0)

# 학습셋 C/D 샘플 및 C vs D 타깃
mask_train_CD = mask_train_notE & (y_train.isin(['C','D']))
X_train_CD    = X_train_s[mask_train_CD]
y_train_CD    = y_train[mask_train_CD].apply(lambda s: 1 if s == 'D' else 0)

# XGBoost 분류기
model4 = XGBClassifier(eval_metric='logloss', random_state=42, verbose=-1, verbosity=0, tree_method='gpu_hist', predictor = 'gpu_predictor')

# C vs D
model4.fit(X_train_CD, y_train_CD)

# 검증셋 C vs D
y4_pred = model4.predict(X_val_CD)

print("\n=== 모델4: C vs D ===")
print(confusion_matrix(y4_true, y4_pred))
print(classification_report(y4_true, y4_pred, target_names=['C','D']))


=== 모델4: C vs D ===
[[2594 3758]
 [ 823 6026]]
              precision    recall  f1-score   support

           C       0.76      0.41      0.53      6352
           D       0.62      0.88      0.72      6849

    accuracy                           0.65     13201
   macro avg       0.69      0.64      0.63     13201
weighted avg       0.68      0.65      0.63     13201



#### 진짜 C/D 인 37개 샘플에 대해서만 모델4를 돌려본 것

In [ ]:
# 실제 C/D 레이블인 모든 검증셋 샘플
df_val_trueCD = df_val_notE[df_val_notE['true_seg'].isin(['C','D'])].reset_index(drop=True)
X_val_trueCD  = df_val_trueCD[X.columns]

# C vs D 레이블 (D→1 / C→0)
y4_true_all   = df_val_trueCD['true_seg'].apply(lambda s: 1 if s == 'D' else 0)

# model4로 전체 C/D 샘플 예측
y4_pred_all   = model4.predict(X_val_trueCD)

# 결과 출력
print("=== 모델4 standalone: 실제 C/D 전체 평가 ===")
print(confusion_matrix(y4_true_all, y4_pred_all))
print(classification_report(y4_true_all, y4_pred_all, labels=[0,1], target_names=['C','D']))

=== 모델4 standalone: 실제 C/D 전체 평가 ===
[[2298 1451]
 [ 824 6026]]
              precision    recall  f1-score   support

           C       0.74      0.61      0.67      3749
           D       0.81      0.88      0.84      6850

    accuracy                           0.79     10599
   macro avg       0.77      0.75      0.76     10599
weighted avg       0.78      0.79      0.78     10599



### 가중치 있는 버전

In [ ]:
# 데이터 로드 및 ID 제거
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FinalProject/tmp폴더/pca_features_with_segment_2.csv")
X = df.drop(columns=['Segment'])
y = df['Segment']

# E vs not-E 이진 타깃 생성
y_e = (y == 'E').astype(int)

# 학습/검증 분할 (20% 검증, E 비율 유지)
X_train, X_val, y_train_e, y_val_e, y_train, y_val = train_test_split(
    X, y_e, y, test_size=0.2, random_state=42, stratify=y_e
)

# 스케일러 학습 및 변환
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s   = scaler.transform(X_val)

In [ ]:
# 1. 모델1: E vs not-E 분류 (가중치 있는)
# pos/neg 비율로 weight 설정
n_neg1 = (y_train_e == 0).sum()
n_pos1 = (y_train_e == 1).sum()
w1 = n_neg1 / n_pos1

# XGBoost with weight
model1_w = CatBoostClassifier(verbose=0, random_seed=42, task_type='GPU')
# 학습·예측
model1_w.fit(X_train_s, y_train_e)
y1_pred_w = model1_w.predict(X_val_s)

print("=== 모델1 (weighted): E vs not-E ===")
print(confusion_matrix(y_val_e, y1_pred_w))
print(classification_report(y_val_e, y1_pred_w, labels=[0,1], target_names=['not-E','E']))

=== 모델1 (weighted): E vs not-E ===
[[10644  5288]
 [ 2575 61493]]
              precision    recall  f1-score   support

       not-E       0.81      0.67      0.73     15932
           E       0.92      0.96      0.94     64068

    accuracy                           0.90     80000
   macro avg       0.86      0.81      0.84     80000
weighted avg       0.90      0.90      0.90     80000



In [ ]:
# 검증셋 DataFrame 및 pred1 추가
df_val = pd.DataFrame(X_val_s, columns=X.columns)
df_val['true_seg'] = y_val.values
df_val['pred1']    = y1_pred_w

# 2. 모델2: A/B vs C/D 분류
df_val_notE = df_val[df_val['pred1']==0].reset_index(drop=True)
X_val_notE  = df_val_notE[X.columns]
y2_true     = df_val_notE['true_seg'].apply(lambda s: 1 if s in ['A','B'] else 0)

mask_train_notE = (y_train_e == 0)
X_train_notE    = X_train_s[mask_train_notE]
y_train_notE    = y_train[mask_train_notE].apply(lambda s: 1 if s in ['A','B'] else 0)

n_neg2 = (y_train_notE == 0).sum()
n_pos2 = (y_train_notE == 1).sum()
w2 = n_neg2 / n_pos2

model2_w = CatBoostClassifier(verbose=0, random_seed=42, task_type='GPU')
model2_w.fit(X_train_notE, y_train_notE)
y2_pred_w = model2_w.predict(X_val_notE)

print("\n=== 모델2 (weighted): A/B vs C/D ===")
print(confusion_matrix(y2_true, y2_pred_w))
print(classification_report(y2_true, y2_pred_w, labels=[0,1], target_names=['C/D','A/B']))

df_val_notE['pred2'] = y2_pred_w


=== 모델2 (weighted): A/B vs C/D ===
[[13181     1]
 [   34     3]]
              precision    recall  f1-score   support

         C/D       1.00      1.00      1.00     13182
         A/B       0.75      0.08      0.15        37

    accuracy                           1.00     13219
   macro avg       0.87      0.54      0.57     13219
weighted avg       1.00      1.00      1.00     13219



In [ ]:
# 3. 모델3: A vs B 분류
df_val_AB = df_val_notE[df_val_notE['pred2']==1].reset_index(drop=True)
X_val_AB  = df_val_AB[X.columns]
y3_true   = df_val_AB['true_seg'].apply(lambda s: 1 if s=='B' else 0)

mask_train_AB = mask_train_notE & (y_train.isin(['A','B']))
X_train_AB    = X_train_s[mask_train_AB]
y_train_AB    = y_train[mask_train_AB].apply(lambda s: 1 if s=='B' else 0)

n_neg3 = (y_train_AB == 0).sum()
n_pos3 = (y_train_AB == 1).sum()
w3 = n_neg3 / n_pos3

model3_w = CatBoostClassifier(verbose=0, random_seed=42, task_type='GPU')
model3_w.fit(X_train_AB, y_train_AB)
y3_pred_w = model3_w.predict(X_val_AB)

print("\n=== 모델3 (weighted): A vs B ===")
print(confusion_matrix(y3_true, y3_pred_w))
print(classification_report(y3_true, y3_pred_w, labels=[0,1], target_names=['A','B']))


=== 모델3 (weighted): A vs B ===
[[4]]
              precision    recall  f1-score   support

           A       1.00      1.00      1.00         4
           B       0.00      0.00      0.00         0

    accuracy                           1.00         4
   macro avg       0.50      0.50      0.50         4
weighted avg       1.00      1.00      1.00         4



In [ ]:
# 4. 모델4: C vs D 분류
df_val_CD = df_val_notE[df_val_notE['pred2']==0].reset_index(drop=True)
X_val_CD  = df_val_CD[X.columns]
y4_true   = df_val_CD['true_seg'].apply(lambda s: 1 if s=='D' else 0)

mask_train_CD = mask_train_notE & (y_train.isin(['C','D']))
X_train_CD    = X_train_s[mask_train_CD]
y_train_CD    = y_train[mask_train_CD].apply(lambda s: 1 if s=='D' else 0)

n_neg4 = (y_train_CD == 0).sum()
n_pos4 = (y_train_CD == 1).sum()
w4 = n_neg4 / n_pos4

model4_w = XGBClassifier(eval_metric='logloss',random_state=42, scale_pos_weight=w4, verbose=-1, verbosity=0, tree_method='gpu_hist', predictor = 'gpu_predictor')
model4_w.fit(X_train_CD, y_train_CD)
y4_pred_w = model4_w.predict(X_val_CD)

print("\n=== 모델4 (weighted): C vs D ===")
print(confusion_matrix(y4_true, y4_pred_w))
print(classification_report(y4_true, y4_pred_w, labels=[0,1], target_names=['C','D']))


=== 모델4 (weighted): C vs D ===
[[3379 2980]
 [1569 5287]]
              precision    recall  f1-score   support

           C       0.68      0.53      0.60      6359
           D       0.64      0.77      0.70      6856

    accuracy                           0.66     13215
   macro avg       0.66      0.65      0.65     13215
weighted avg       0.66      0.66      0.65     13215

